In [21]:
from __future__ import print_function

import argparse

import os
import sys
import pandas as pd
import torch
from torch.utils import data
from torchvision import transforms
from PIL import Image
import cv2
import  glob
import time
import albumentations
import pandas as pd
import numpy as np
from collections import Counter

from sklearn.preprocessing import OneHotEncoder# creating instance of one-hot-encoder
from sklearn.model_selection import train_test_split
### Internal Imports
from models.ResNext50 import Myresnext50
from train.train_classification import trainer_classification
from utils.utils import configure_optimizers
from Datasets.DataLoader import Img_DataLoader

### PyTorch Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import glob

In [22]:
# Load the data, split it into training and validation dataframes
df = pd.read_pickle('imagepaths.pkl')

train_df, val_df = train_test_split(df, test_size=0.15, random_state=42, stratify=df['Label'])
print(f"Training set shape: {train_df.shape}          Training set label count: {str(Counter(train_df['Label'].to_list()))[7:][1:][:-1]} \n")
print(f"Validation set shape: {val_df.shape}         Validation set label count: {str(Counter(val_df['Label'].to_list()))[7:][1:][:-1]} \n")

Training set shape: (17669, 4)          Training set label count: {'Other': 10069, 'Myeloid': 6582, 'Lymphoid': 1018} 

Validation set shape: (3119, 4)         Validation set label count: {'Other': 1777, 'Myeloid': 1162, 'Lymphoid': 180} 



In [39]:
# Load filepaths
X_train = train_df['Filepath'].to_list()
X_val = str(val_df['Filepath'].to_list())

# Load df that represents the one hot encoding of each cell type (Myeloid, Lymphoid, other)
cell_types_df = pd.read_pickle("cell_types_df.pkl")

# Load model
resnext50_pretrained = torch.hub.load('pytorch/vision:v0.10.0', 'resnext50_32x4d', pretrained=True)
my_extended_model = Myresnext50(my_pretrained_model= resnext50_pretrained, num_classes = 3)

Using cache found in C:\Users\moone/.cache\torch\hub\pytorch_vision_v0.10.0
C:\Users\moone\miniconda3\envs\thesis\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\moone\miniconda3\envs\thesis\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt50_32X4D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [46]:
X_train[0].split('_')[0].split("\\")[-1].replace(" ", "")

'AML'

In [33]:
# Simple augumentation to improve the data generalibility

transform_pipeline = albumentations.Compose(
    [
        albumentations.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),

    ]
)

In [44]:
# Define training setup
trainer = trainer_classification(train_image_files=X_train, validation_image_files=X_val, model=my_extended_model,
                                     img_transform=transform_pipeline, init_lr=0.001,
                                     lr_decay_every_x_epochs=10,

                                     weight_decay=0.0005, batch_size=1024, epochs=30, gamma=0.1, df=cell_types_df,
                                     save_checkpoints_dir='checkpoints')

In [45]:
# Train the model
My_model = trainer.train(my_extended_model)

==> Create model.
==> List learnable parameters
==> Load data.
17669
372663
==> Configure optimizer.
10
==> Start training
==> Creat the saving dictionary
Created new dictionary for saving checkpoints


IndexError: Caught IndexError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "C:\Users\moone\miniconda3\envs\thesis\lib\site-packages\torch\utils\data\_utils\worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
  File "C:\Users\moone\miniconda3\envs\thesis\lib\site-packages\torch\utils\data\_utils\fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "C:\Users\moone\miniconda3\envs\thesis\lib\site-packages\torch\utils\data\_utils\fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "D:\Mathijs\Open Universiteit\Thesis\Implementation\DeepHeme_training\Datasets\DataLoader.py", line 44, in __getitem__
    label = img_path.split('/')[-2]
IndexError: list index out of range
